In [ ]:
# default_exp train_loop

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# hide
from fastcore.test import *

# Training Loop

> Define the general Training Loop

In [ ]:
# export 

# general
import torch
from tqdm import tqdm

# import the relevant models
from deeptool.model.dcgan import DCGAN
from deeptool.model.diagnosis import TripleMRNet
from deeptool.model.vqvae import VQVAE2
from deeptool.model.introvae import IntroVAE
from deeptool.model.rnnvae import *

# import the dataset
from deeptool.dataloader import load_datasets

# import the parameters
from deeptool.parameters import get_all_args

In [ ]:
# export

def get_model(device, args):
    """
    return the required model depending on the arguments:
    """
    switcher = {
        "dcgan": DCGAN,
        "diagnosis": TripleMRNet,
        "vqvae": VQVAE2,
        "introvae": IntroVAE,
        # ToDO
    }
    # Get the model_creator
    model_creater = switcher.get(args.model_type, lambda: "Invalid Model Type")
    # create model
    return model_creater(device, args)

In [ ]:
args = get_all_args()
device = torch.device("cuda:0" if (torch.cuda.is_available() and args.n_gpu > 0) else "cpu")
model = get_model(device, args)

In [ ]:
# export

def test_dataset(args, valid_loader):
    """
    return the required model depending on the arguments:
    """
    test_data = next(iter(valid_loader))
    print("Input dimension:")

    if args.model_type == "triplenet":
        print(test_data["img"][args.perspectives[0]].shape)
        print(torch.min(test_data["img"][args.perspectives[0]]))
        print(torch.max(test_data["img"][args.perspectives[0]]))
    else:
        print(test_data["img"].shape)
        print(torch.min(test_data["img"]))
        print(torch.max(test_data["img"]))

In [ ]:
# export

def main(args):
    """Perform the Training using the predefined arguments"""

    # define calculating device
    device = torch.device("cuda:0" if (
        torch.cuda.is_available() and args.n_gpu > 0) else "cpu")

    # build the dataloaders
    train_data, _, train_loader, valid_loader = load_datasets(
        args)

    # test dataset
    test_dataset(args, valid_loader)

    # Initialise the Model
    model = get_model(device, args)

    # Load pretrained params
    if args.load_model:
        model.load_state_dict(torch.load(args.model_path))

    # start training
    print("Start training")
    batch_count = 0
    model.train()

    for epoch in range(args.n_epochs):

        # remodify after pretraining training
        if epoch == args.n_pretrain and args.model_type in ("vqvae", "introvae"):
            print("parameters set")
            model.set_parameters(args)

        # For each batch in the dataloader
        for data in tqdm(train_loader):
            # perform training
            model(data)

            if batch_count % args.watch_batch == 0:
                # eval status
                model.eval()

                if args.model_type in ("triplenet", "diagnosis"):
                    model.watch_progress(valid_loader, epoch)
                else:
                    model.watch_progress(test_data, epoch)

                # reset to train status again
                model.train()

            # watch progress
            batch_count += 1

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_rnn_vae.ipynb.
Converted 99_index.ipynb.
